# Изучаем закономерности в данных

## Задание 4.1
```sql
select
    a.city
from
    dst_project.airports a
group by
    a.city
having
    count(a.airport_code) > 1
```

## Задание 4.2

**Вопрос 1.**

```sql
select
    count(distinct fl.status)
from
    dst_project.flights fl
```

**Вопрос 2.**
```sql
select
    count(fl.flight_id)
from
    dst_project.flights fl
where
    fl.status = 'Departed'
```

**Вопрос 3.**
```sql
select 
    count(s.seat_no)
from 
    dst_project.aircrafts a
        join dst_project.seats s 
        on a.aircraft_code = s.aircraft_code
where
    a.aircraft_code = '773'
```

**Вопрос 4.**
```sql
select 
    count(fl.flight_id)
from 
    dst_project.flights fl
where
    fl.status = 'Arrived'
    and fl.actual_arrival between '2017-04-01' and '2017-09-01'
```

## Задание 4.3 

**Вопрос 1.**
```sql
select
    count(f.flight_id)
from 
    dst_project.flights f 
where
    f.status = 'Cancelled'
```

**Вопрос 2.**
```sql
select
    'Boeing' model,
    count(a.model) quantity
from
    dst_project.aircrafts a
where
    a.model like 'Boeing%'
union all
select
    'Sukhoi Superjet' model,
    count(a.model) quantity
from
    dst_project.aircrafts a
where
    a.model like 'Sukhoi Superjet%'
union all
select
    'Airbus' model,
    count(a.model) quantity
from
    dst_project.aircrafts a
where
    a.model like 'Airbus%'
order by 
    quantity desc
```

**Вопрос 3.**
```sql
select
    'Asia' timezone,
    count(a.timezone) airports
from
    dst_project.airports a
where
    a.timezone like 'Asia%'
union all
select
    'Europe' timezone,
    count(a.timezone) airports
from
    dst_project.airports a
where
    a.timezone like 'Europe%'
union all
select
    'Australia' timezone,
    count(a.timezone) airports
from
    dst_project.airports a
where
    a.timezone like 'Australia%'
union all
select
    'Total' timezone,
    count(a.timezone) airports
from
    dst_project.airports a
order by
    2 desc
```

**Вопрос 4.**
```sql
select
    f.flight_id,
    f.actual_departure - f.scheduled_departure
from
    dst_project.flights
where
    status = 'Arrived'
group by
    f.flight_id
order by 
    2 desc
limit
    1
```

## Задание 4.4

**Вопрос 1.**
```sql
select
    min(f.scheduled_departure)
from
    dst_project.flights f
```

**Вопрос 2.**
```sql
select
    f.scheduled_arrival - f.scheduled_departure
from
    dst_project.flights f
where
    status = 'Arrived'
order by 
    1 desc
```

**Вопрос 3.**
```sql
select
    f.arrival_airport,
    f.departure_airport,
    f.scheduled_arrival - f.scheduled_departure duration
from
    dst_project.flights f
where
    status = 'Scheduled'
order by 
    3 desc
```

**Вопрос 4.**
```sql
select
    avg(f.actual_arrival - f.actual_departure) duration
from
    dst_project.flights f
where
    status = 'Arrived'
```

## Задание 4.5

**Вопрос 1.**
```sql
select
    distinct s.fare_conditions,
    count(s.seat_no)
from
    dst_project.seats s
where 
    s.aircraft_code = 'SU9'
group by
    fare_conditions
order by
    2 desc
```

**Вопрос 2.**
```sql
select
    min(b.total_amount)
from
    dst_project.bookings b 
```

**Вопрос 3.**
```sql
select
    b.seat_no
from
    dst_project.boarding_passes b
        left join dst_project.tickets t
        on b.ticket_no = t.ticket_no
where
    t.passenger_id = '4313 788533'
```

# Предварительный анализ

## Задание 5.1

**Вопрос 1.**
```sql
/* Достаем код аэропорта Анапы */
select
    a.airport_code
from
    dst_project.airports a
where
    a.city = 'Anapa'

/* Нас интересует AAQ */
select
    count(f.flight_id)
from
    dst_project.flights f
where
    f.arrival_airport = 'AAQ'
    and extract (year from f.actual_departure) = 2017
group by 
    f.arrival_airport
```

**Вопрос 2.**
```sql
select
    count(f.flight_id)
from
    dst_project.flights f
where
    f.departure_airport = 'AAQ'
    and (date_trunc('month', scheduled_departure) in ('2017-01-01','2017-02-01', '2017-12-01'))
```

**Вопрос 3.**
```sql
select
    count(f.flight_id)
from
    dst_project.flights f
where
    f.departure_airport = 'AAQ'
    and f.status = 'Cancelled'
```

**Вопрос 4.**
```sql
select
    count(f.flight_id)
from
    dst_project.flights f
where
    f.departure_airport = 'AAQ'
    and f.arrival_airport != 'SVO' -- это единственный аэропорт в Москве, в который прилетают рейсы из Анапы
```

**Вопрос 5.**
```sql
select 
    a.aircraft_code, 
    a.model, 
    count(distinct s.seat_no)
from 
    dst_project.seats s
        join dst_project.aircrafts a 
        on s.aircraft_code = a.aircraft_code
        join dst_project.flights f
        on s.aircraft_code = f.aircraft_code
where 
    f.departure_airport = 'AAQ' 
group by 
    1, 2 
order by 
    3 desc 
limit 
    1 
```

# Выгрузка датасета

```sql
-- Данные о полетах и самолетах
with f as (                 
          select f.flight_id,
          f.departure_airport,
          f.arrival_airport,
          (date_part('hour', f.actual_arrival - f.actual_departure) * 60 + date_part('minute', f.actual_arrival - f.actual_departure)) flight_duration,
          f.aircraft_code
   from dst_project.flights as f
   where f.departure_airport = 'AAQ'
     and (date_trunc('month', f.scheduled_departure) in ('2017-01-01',
                                                         '2017-02-01',
                                                         '2016-12-01'))
     and f.status in ('Arrived')
),
-- Данные о билетах и прибыли
     t as (
     select t.flight_id,
          count(t.ticket_no) tickets_amount,
          sum(t.amount) revenue
     from dst_project.ticket_flights as t
     group by 1
     ),
-- Данные о классе обслуживания
    tc as (
     select tf.flight_id,
         count(case when tf.fare_conditions = 'Economy' then tf.fare_conditions end) economy_class,
         count(case when tf.fare_conditions = 'Comfort' then tf.fare_conditions end) comfort_class,
         count(case when tf.fare_conditions = 'Business' then tf.fare_conditions end) business_class
     from dst_project.ticket_flights as tf
     group by 1
     )
select f.flight_id,
       f.departure_airport,
       f.arrival_airport,
       a.model,
       t.tickets_amount,
       t.revenue,
       tc.economy_class,
       tc.comfort_class,
       tc.business_class
from dst_project.flights fl
    left join f on fl.flight_id = f.flight_id
    left join t on fl.flight_id = t.flight_id
    left join tc on fl.flight_id = tc.flight_id
    left join dst_project.aircrafts a on fl.aircraft_code = a.aircraft_code
where fl.departure_airport = 'AAQ'
    and (date_trunc('month', scheduled_departure) in ('2017-01-01', '2017-02-01', '2017-12-01'))
    and status not in ('Cancelled')
order by 1
```